In [1]:
import numpy as np
import matplotlib.pyplot as plt
from reusable import Auction, MultiplicativePacingAgent, UCBLikeAgent, EXP3AgentPrimalDual



# Requirement 4

In [2]:
class FirstPriceAuction(Auction):
    def __init__(self, ctrs):
        self.ctrs = ctrs
        self.n_adv = len(self.ctrs)
    
    def get_winners(self, bids):
        # we compute the expected utility of each ad for the advertiser
        adv_values = self.ctrs * bids

        adv_ranking = np.argsort(adv_values)

        winner = adv_ranking[-1]
        return winner, adv_values
    
    def get_payments_per_click(self, winners, values, bids):
        # In a first-price auction, the winner pays their bid amount
        payment = bids[winners]
        return round(payment, 2)

In [3]:
def run_simulation(n_advertisers, n_rounds, budget, base_valuation):
    ctrs = np.random.uniform(0.1, 1, n_advertisers)  # Random CTRs for each advertiser
    auction = FirstPriceAuction(ctrs)
    wins = [0 for i in range(n_advertisers)]
    
    agents = [
        MultiplicativePacingAgent(valuation=base_valuation, budget=budget, T=n_rounds, eta=0.1),
        MultiplicativePacingAgent(valuation=base_valuation, budget=budget, T=n_rounds, eta=0.1),
        UCBLikeAgent(budget=budget, T=n_rounds, bids=np.linspace(0, base_valuation, 100)),
        UCBLikeAgent(budget=budget, T=n_rounds, bids=np.linspace(0, base_valuation, 100)),
        EXP3AgentPrimalDual(num_slots=10, num_prices=10, learning_rate=0.1),
        EXP3AgentPrimalDual(num_slots=10, num_prices=10, learning_rate=0.1)
    ]
    
    rewards = {i: [] for i in range(n_advertisers)}
    cumulative_rewards = {i: [0] for i in range(n_advertisers)}

    for round in range(n_rounds):
        bids = [agent.bid() for agent in agents]
        
        winner, values = auction.get_winners(bids)
        payment = auction.get_payments_per_click(winner, values, bids)
        
        for i, agent in enumerate(agents):
            if i == winner:
                wins[i] += 1
                reward = ctrs[i] * (base_valuation - payment)
                agent.update(reward, payment)
                rewards[i].append(reward)
                cumulative_rewards[i].append(cumulative_rewards[i][-1] + reward)
            else:
                agent.update(0, 0)
                rewards[i].append(0)
                cumulative_rewards[i].append(cumulative_rewards[i][-1])

    return rewards, cumulative_rewards, wins

# Run the simulation
n_advertisers = 5
n_rounds = 10000
budget = 1000
base_valuation = 1

rewards, cumulative_rewards, wins = run_simulation(n_advertisers, n_rounds, budget, base_valuation)

# Plot results
plt.figure(figsize=(12, 6))
for i in range(len(cumulative_rewards)):
    plt.plot(cumulative_rewards[i], label=f'Agent {i}')

plt.title('Cumulative Reward over Time (First Price Auction)')
plt.xlabel('Rounds')
plt.ylabel('Cumulative Reward')
plt.legend()
plt.show()

AttributeError: 'EXP3AgentPrimalDual' object has no attribute 'bid'

In [ ]:
# Print number of wins and cumulative utilities for each agent
print("Agent Results:")
print("Agent\tWins\tCumulative Utility")
print("-----\t----\t------------------")
for i in range(n_advertisers):
    print(f"{i}\t{wins[i]}\t{cumulative_rewards[i][-1]:.2f}")

# Print total wins
total_wins = sum(wins)
print(f"\nTotal wins: {total_wins}")

# Print win percentages
print("\nWin Percentages:")
for i in range(n_advertisers):
    win_percentage = (wins[i] / total_wins) * 100
    print(f"Agent {i}: {win_percentage:.2f}%")

Agent Results:
Agent	Wins	Cumulative Utility
-----	----	------------------
0	2193	186.69
1	7395	3760.90
2	19	-452.10
3	14	-181.92
4	379	-425.37

Total wins: 10000

Win Percentages:
Agent 0: 21.93%
Agent 1: 73.95%
Agent 2: 0.19%
Agent 3: 0.14%
Agent 4: 3.79%
